In [1]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from transformers import TFGPT2LMHeadModel,GPT2Tokenizer
from datasets import Dataset
import pandas as pd


2025-02-15 20:33:51.233973: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-15 20:33:51.254405: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-15 20:33:51.270626: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-15 20:33:51.275182: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-15 20:33:51.292116: I tensorflow/core/platform/cpu_feature_guar

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

I0000 00:00:1739640834.176165    9655 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1739640834.177641    9655 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1739640834.177695    9655 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
from tensorflow.keras import mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)


In [4]:
df=pd.read_csv('trip-data.csv')

In [5]:
df

,id,Activity,location,Description,Type,Price,openig-hours,holiday,act_time,time
0,0,Sharjah Archaeology Museum,Sharjah City,Ancient treasures and artifacts from UAE's pas...,museum,low,Saturday to Thursday:8:00 20:00 Friday:16:00 2...,0,8:00 20:00,"['morning','afternoon','evening']"
1,1,Bait Al Naboodah Museum,Sharjah City,"Historic Gulf home with coral walls, intricate...",museum,low,Saturday to Thursday:8:00 20:00 Friday:16:00 2...,0,8:00 20:00,"['morning','afternoon','evening']"
2,2,Bait Khalid Bin Ibrahim,Sharjah City,Historic house turned boutique hotel offers au...,old_hotel,low,Saturday to Thursday: 8:00 20:00 Friday: 16:00...,0,8:00 20:00,"['morning','afternoon','evening']"
3,3,Sharjah Calligraphy Museumc,Sharjah City,Arab world's only museum dedicated to calligra...,museum,low,Saturday to Thursday: 8:00 20:00 Friday: 16:00...,0,8:00 20:00,"['morning','afternoon','evening']"
4,4,Cultural Square,Sharjah City,Cultural Square: 14-metre high Quran sculpture...,museum,low,friday to friday:12:00 00:00,0,12:00 00:00,"['morning','afternoon','evening','night']"
...,...,...,...,...,...,...,...,...,...,...
1132,1182,Kalba Beach Corniche,East Coast,Explore the Kalba Beach Corniche in East Coast...,trendy,high,Monday to Friday: 9:00 AM - 6:00 PM,0,9:00 AM - 6:00 PM,"['morning', 'afternoon', 'evening']"
1133,1183,Bait Al Naboodah Museum,Sharjah City,Explore the Bait Al Naboodah Museum in Sharjah...,precious,medium,Monday to Friday: 9:00 AM - 6:00 PM,0,9:00 AM - 6:00 PM,"['morning', 'afternoon', 'evening']"
1134,1184,Najd Al Maqsar Village,Central Region,Explore the Najd Al Maqsar Village in Central ...,lively,medium,Monday to Friday: 9:00 AM - 6:00 PM,0,9:00 AM - 6:00 PM,"['morning', 'afternoon', 'evening']"
1135,1185,Kalba Birds of Prey Centre,East Coast,Explore the Kalba Birds of Prey Centre in East...,bustling,medium,Monday to Friday: 9:00 AM - 6:00 PM,0,9:00 AM - 6:00 PM,"['morning', 'afternoon', 'evening']"


In [6]:
df['input_text'] = df['location']+""+df['Type']
df['target_text'] = df['Description']

In [7]:
df['input_text'].str.len().max()

41

In [8]:
df['target_text'].str.len().max()

163

In [9]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [10]:
train_dataset = Dataset.from_pandas(train_df[['input_text', 'target_text']])
test_dataset = Dataset.from_pandas(test_df[['input_text', 'target_text']])

In [11]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = TFGPT2LMHeadModel.from_pretrained("gpt2")

I0000 00:00:1739640838.908680    9655 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1739640838.908767    9655 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1739640838.908792    9655 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1739640839.125322    9655 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1739640839.125386    9655 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-02-15

In [12]:
tokenizer.pad_token_id=tokenizer.eos_token_id

In [13]:
def tokenize_function(examples):
    input_texts = [str(text) if isinstance(text, str) else "" for text in examples["input_text"]]
    target_texts = [str(text) if isinstance(text, str) else "" for text in examples["target_text"]]
    model_inputs = tokenizer(input_texts, padding="max_length", truncation=True, max_length=163, return_tensors="tf")
    labels = tokenizer(target_texts, padding="max_length", truncation=True, max_length=163, return_tensors="tf")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)
def to_tf_dataset(dataset):
    return tf.data.Dataset.from_tensor_slices((
        {
            'input_ids': dataset['input_ids'],
            'attention_mask': dataset['attention_mask'],
        },
        dataset['labels']
    ))
train_tf_dataset = to_tf_dataset(train_dataset).batch(8)
test_tf_dataset = to_tf_dataset(test_dataset).batch(8)

Map:   0%|          | 0/909 [00:00<?, ? examples/s]

Map:   0%|          | 0/228 [00:00<?, ? examples/s]

In [19]:
train_tf_dataset = to_tf_dataset(train_dataset).batch(4)
test_tf_dataset = to_tf_dataset(test_dataset).batch(4)


In [14]:
from keras.optimizers import Adam
model.compile(optimizer='adam', 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])

In [15]:
model.fit(train_tf_dataset, validation_data=test_tf_dataset, epochs=15)


Epoch 1/15


I0000 00:00:1739640867.838206    9775 service.cc:146] XLA service 0x7f8529442a40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1739640867.838264    9775 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-02-15 20:34:27.844565: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-02-15 20:34:27.855485: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8902
I0000 00:00:1739640867.890497    9775 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


114/114 [==============================] - 39s 230ms/step - loss: 0.9341 - accuracy: 0.8696 - val_loss: 0.5267 - val_accuracy: 0.8940
Epoch 2/15
114/114 [==============================] - 25s 221ms/step - loss: 0.5023 - accuracy: 0.8947 - val_loss: 0.4690 - val_accuracy: 0.8986
Epoch 3/15
114/114 [==============================] - 25s 221ms/step - loss: 0.4572 - accuracy: 0.8987 - val_loss: 0.4530 - val_accuracy: 0.9010
Epoch 4/15
114/114 [==============================] - 25s 221ms/step - loss: 0.4338 - accuracy: 0.9011 - val_loss: 0.4463 - val_accuracy: 0.9016
Epoch 5/15
114/114 [==============================] - 25s 221ms/step - loss: 0.4140 - accuracy: 0.9027 - val_loss: 0.4482 - val_accuracy: 0.9018
Epoch 6/15
114/114 [==============================] - 25s 221ms/step - loss: 0.4023 - accuracy: 0.9038 - val_loss: 0.4438 - val_accuracy: 0.9020
Epoch 7/15
114/114 [==============================] - 25s 221ms/step - loss: 0.3923 - accuracy: 0.9041 - val_loss: 0.4425 - val_accuracy: 0.9

In [16]:
model.fit(train_tf_dataset, validation_data=test_tf_dataset, epochs=30)


Epoch 1/30
114/114 [==============================] - 25s 221ms/step - loss: 0.3026 - accuracy: 0.9119 - val_loss: 0.5337 - val_accuracy: 0.8986
Epoch 2/30
114/114 [==============================] - 25s 221ms/step - loss: 0.2887 - accuracy: 0.9141 - val_loss: 0.5371 - val_accuracy: 0.8980
Epoch 3/30
114/114 [==============================] - 25s 221ms/step - loss: 0.2815 - accuracy: 0.9160 - val_loss: 0.5471 - val_accuracy: 0.8953
Epoch 4/30
114/114 [==============================] - 25s 221ms/step - loss: 0.2754 - accuracy: 0.9168 - val_loss: 0.5732 - val_accuracy: 0.8987
Epoch 5/30
114/114 [==============================] - 25s 222ms/step - loss: 0.2674 - accuracy: 0.9185 - val_loss: 0.5806 - val_accuracy: 0.8967
Epoch 6/30
114/114 [==============================] - 25s 222ms/step - loss: 0.2629 - accuracy: 0.9195 - val_loss: 0.5794 - val_accuracy: 0.8971
Epoch 7/30
114/114 [==============================] - 25s 222ms/step - loss: 0.2577 - accuracy: 0.9204 - val_loss: 0.6224 - val_ac